In [1]:
import json

In [47]:
original = json.load(open("data/verses_clauses_dict.json", "r", encoding="utf-8"))
results = [json.loads(line) for line in open("data/fine_tuning_datasets/trial_1/model_outputs/output_validation_small_temp_0.jsonl", encoding="utf-8").readlines()]

In [48]:
def extract_result(line):
    return line["verse"].split(":")[1].strip(), line["parsed_clauses"].replace("```", "").replace("json", "").replace("\n", "")

In [49]:
len(results)

200

In [50]:
#extract_result(a)

In [51]:
def extract_results(results):
    verse_clauses = {}
    for result in results:
        verse, clauses = extract_result(result)
        verse_clauses[verse] = clauses
    return verse_clauses

In [52]:
verse_clauses_results = extract_results(results)

In [53]:
#verse_clauses_results

In [54]:
len(verse_clauses_results)

200

In [55]:
nb_total = 0
nb_correct = 0

nb_simple_total = 0
nb_simple_correct = 0

nb_complex_total = 0
nb_complex_correct = 0

nb_clauses_total = 0
nb_clauses_correct = 0

nb_complex_clauses_total = 0
nb_complex_clauses_correct = 0

nb_simple_clauses_total = 0
nb_simple_clauses_correct = 0

for verse, clauses in verse_clauses_results.items():
    nb_total += 1
    true_clauses = original[verse]["clauses"]
    nb_clauses_total += len(true_clauses)
    complexity = original[verse]["complexity"]
    if complexity == "simple":
        nb_simple_clauses_total += len(true_clauses)
        nb_simple_total += 1
    else:
        nb_complex_clauses_total += len(true_clauses)
        nb_complex_total += 1
        
    try:
        clauses = json.loads(clauses)
    except json.JSONDecodeError:
        print("wrong json", clauses)
        continue
    complexity = original[verse]["complexity"]
    print("res", clauses)
    print("cor", true_clauses, complexity)
    print(f" Model result is {clauses == true_clauses}")
    print("")

    for clause in clauses:
        if clause in true_clauses:
            nb_clauses_correct += 1
            
            if complexity == "simple":
                nb_simple_clauses_correct += 1
            else:
                nb_complex_clauses_correct += 1
    
    if clauses == true_clauses:
        nb_correct += 1
        if complexity == "simple":
            nb_simple_correct += 1
        else:
            nb_complex_correct += 1

res ['ויאמר שמואל אלשאול', 'אתי שלח יהוה', 'למשחך למלך עלעמו עלישראל', 'ועתה שמע לקול דברי יהוה ס']
cor ['ויאמר שמואל אלשאול', 'אתי שלח יהוה', 'למשחך למלך עלעמו עלישראל', 'ועתה', 'שמע לקול דברי יהוה ס'] simple
 Model result is False

res ['וימת חזאל מלךארם', 'וימלך בןהדד בנו תחתיו']
cor ['וימת חזאל מלךארם', 'וימלך בןהדד בנו תחתיו'] simple
 Model result is True

res ['ועל ראש העמודים מעשה שושן', 'ותתם מלאכת העמודים']
cor ['ועל ראש העמודים מעשה שושן', 'ותתם מלאכת העמודים'] simple
 Model result is True

res ['וישב ארצו ברכוש גדול', 'ולבבו עלברית קדש', 'ועשה', 'ושב לארצו']
cor ['וישב ארצו ברכוש גדול', 'ולבבו עלברית קדש', 'ועשה', 'ושב לארצו'] simple
 Model result is True

res ['ורחצו ממנו משה ואהרן ובניו אתידיהם ואתרגליהם']
cor ['ורחצו ממנו משה ואהרן ובניו אתידיהם ואתרגליהם'] simple
 Model result is True

res ['וזה הדבר', 'אשר תעשו', 'כלזכר וכלאשה', 'ידעת', 'משכבזכר', 'תחרימו']
cor ['וזה הדבר', 'אשר תעשו', 'כלזכר וכלאשה תחרימו', 'ידעת משכבזכר'] complex
 Model result is False

res ['ומנחתם ס

In [56]:
print(f"Correct parsing: {nb_correct} over {nb_total}.")
print(f"Percentage of correct parsing: {nb_correct / nb_total * 100} %")

Correct parsing: 96 over 200.
Percentage of correct parsing: 48.0 %


In [57]:
print(f"Correct parsing of simple verses: {nb_simple_correct} over {nb_simple_total}.")
print(f"Percentage of correct parsing for simple verses: {nb_simple_correct / nb_simple_total * 100} %")

Correct parsing of simple verses: 96 over 179.
Percentage of correct parsing for simple verses: 53.63128491620112 %


In [58]:
print(f"Correct parsing of complex verses: {nb_complex_correct} over {nb_complex_total}.")
print(f"Percentage of correct parsing for complex verses: {nb_complex_correct / nb_complex_total * 100} %")

Correct parsing of complex verses: 0 over 21.
Percentage of correct parsing for complex verses: 0.0 %


In [59]:
print(f"Correct parsing of individual clauses: {nb_clauses_correct} over {nb_clauses_total}.")
print(f"Percentage of correct parsing: {nb_clauses_correct / nb_clauses_total * 100} %")

Correct parsing of individual clauses: 554 over 779.
Percentage of correct parsing: 71.1168164313222 %


In [60]:
print(f"Correct parsing of individual clauses in simple verses: {nb_simple_clauses_correct} over {nb_simple_clauses_total}.")
print(f"Percentage of correct parsing in simple verses: {nb_simple_clauses_correct / nb_simple_clauses_total * 100} %")

Correct parsing of individual clauses in simple verses: 518 over 694.
Percentage of correct parsing in simple verses: 74.63976945244957 %


In [61]:
print(f"Correct parsing of individual clauses in complex verses: {nb_complex_clauses_correct} over {nb_complex_clauses_total}.")
print(f"Percentage of correct parsing in complex verses: {nb_complex_clauses_correct / nb_complex_clauses_total * 100} %")

Correct parsing of individual clauses in complex verses: 36 over 85.
Percentage of correct parsing in complex verses: 42.35294117647059 %
